In [2]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import random
sys.path.insert(1, "/Users/xujingyu/2023 fall course/4775/group project/git project /dataset")
sys.path.insert(2, "/Users/xujingyu/2023 fall course/4775/group project/git project /")
import dbm
import dataset_yeast_mini
import fungi_mini
import fungi
import vertebrates
import fungi_large
import vertebrates_large
import gapped
from column_distance import *
from clustering import hierarchical_clustering
from clustering import kmeans_self_defined_dist
from clustering import kmeans_hier_init
from distance_matrix_calculation import pearson_distance
from distance_matrix_calculation import jensen_distance
from distance_matrix_calculation import dist_from_col
from evaluate import ari_score
from evaluate import fmi_score
from evaluate import nmi_score
from evaluate import graph

This program runs on database: JASPAR2024
178.0motifs have been fetched.
there is no overlap between clusters. 
        0      1      2      3      4      5      6      7      8
A:  -1.06   -inf   1.71   -inf  -2.73   -inf  -1.92   1.73   1.70
C:  -0.40   1.89  -1.59   -inf   -inf   -inf   -inf  -2.65  -2.64
G:   1.42   -inf   -inf   -inf   1.94   -inf   1.90  -0.94  -0.75
T:  -3.60  -1.75  -1.31   2.00   -inf   2.00   -inf   -inf   -inf

This program runs on database: JASPAR2024
229.0motifs have been fetched.
there is no overlap between clusters. 
        0      1      2      3      4      5      6      7      8      9
A:   1.07   1.91   1.96   -inf  -7.96  -6.96  -7.96  -6.96   1.66   1.07
C:  -1.55  -5.38   -inf   1.51  -3.96   1.99   1.98  -7.96  -3.96  -1.64
G:   0.20  -4.38  -4.50   0.20   -inf  -7.96  -4.38   1.99  -0.57   0.04
T:  -1.26  -2.64  -3.96  -6.96   1.98  -6.96  -7.96  -6.96  -3.38  -0.86



In [5]:
################################################################################

# Helper Functions


def assign_unique_numbers_to_values(dataset):
    """
    Assign unique numbers to the values in the dataset dictionary.
    If two values are the same, they will be given the same number.

    :param dataset: dict, a dictionary with values that need unique numbers assigned.
    :return: dict, a dictionary with values replaced by unique numbers.
    """
    unique_values = set(dataset.values())  # Get the unique values
    value_to_number = {
        value: idx for idx, value in enumerate(unique_values)
    }  # Map each unique value to a number

    return {key: value_to_number[value] for key, value in dataset.items()}


def generate_labels_from_clusters(clusters):
    """
    Generate a dictionary of labels based on cluster membership.
    Items in the same list receive the same label, starting from 0.

    :param clusters: list of lists, where each sublist represents a cluster.
    :return: dict, a dictionary with items as keys and their respective labels as values.
    """
    label_dict = {}
    for label, cluster in enumerate(clusters):
        for item in cluster:
            label_dict[item] = label
    return label_dict


def create_cluster_dicts(clusters, dataset):
    """
    Create a list of dictionaries for each cluster.

    :param clusters: A list of clusters, each cluster being a list of item keys.
    :param dataset: The dataset containing the items and their corresponding data.
    :return: A list of dictionaries, each dictionary representing a cluster.
    """
    our_cluster = []
    for cluster in clusters:
        cluster_dict = {}
        for item in cluster:
            cluster_dict[item] = dataset.mmm[item]
        our_cluster.append(cluster_dict)
    return our_cluster

def create_random_subdatasets(original_dataset, subdataset_sizes, retain_cluster_structure=False):
    # Flatten the original dataset
    flattened_dataset = [item for cluster in original_dataset for item in cluster]

    subdatasets = []

    for size in subdataset_sizes:
        if size > len(flattened_dataset):
            raise ValueError(f"Requested subdataset size {size} is larger than the total number of elements.")

        # Randomly select elements for the subdataset
        subdataset = random.sample(flattened_dataset, size)

        # Reshape into the original format, if required
        if retain_cluster_structure:
            reshaped_subdataset = []
            temp = subdataset.copy()
            for cluster in original_dataset:
                cluster_size = len(cluster)
                reshaped_subdataset.append(temp[:cluster_size])
                temp = temp[cluster_size:]
            subdataset = reshaped_subdataset

        subdatasets.append(subdataset)

    return subdatasets

import numpy as np
import os


class TreeNode(object):
    """
    Data structure for the hierarchical tree. 
    This object is either an inner node or a leaf. 
    If it is an inner node:
    both left and right attributes should be another treenode object.
    height > 0. name should be "inner#". mn (member number)
    If it is a leaf:
    Both left and right attribute should be None.
    Height = 0. name should be a specific motif id. 
    mn = 1
    """
    def __init__(self, name, left_child, right_child, height, mn):
        self.left = left_child
        self.right = right_child
        self.name = name
        self.mn = mn
        self.height = height
        # if this is an innernode: 
        if self.left != None: 
            assert self.height > 0
            assert self.left.height <= self.height
            assert self.right.height <= self.height


# This algorithm will first build a tree represented by class "TreeNode", then traverse the treenode
# for heights from highest to the lowest until it finds k-1 nodes. After neglecting these top n-1 nodes, 
# the rest tree nodes just represents how to group the motifs into clusters. 

def wrap_dm(idlist, dm):
    """
    helper functions to first wrap the motif idlist into a leaf list, and to convert the distance matrix into a distance map. 
    Precondition: 
        idlist: a list of motif ids. 
        dm: a np array representing the distance matrix. it should have a shape of len(idlist) * len(idlist)
    Returns:
        nodemap: a map of treenodes: treenode.name -> treenode
        distance_map: a map of treenode.name -> treenode.name -> distance
    """
    nodemap = {id : TreeNode(id, None, None, 0, 1) for id in idlist}
    distance_map = {}
    for i in range(len(idlist)):
        motif_id1 = idlist[i]
        id1_map = {}
        for j in range(len(idlist)):
            id1_map[idlist[j]] = dm[i][j]
        distance_map[motif_id1] = id1_map
    return nodemap, distance_map

def shortest(dm):
    """
    helper function that finds the shortest distance between two different nodes in a distance map.
    Returns the shortest distance as well as the two TreeNode.names
    Precondition:
        dm: a map of treenode.name -> treenode.name -> distance
    """
    ret = (None, None, np.inf)
    for key1 in dm.keys():
        for key2 in dm[key1].keys():
            if key1 != key2 and dm[key1][key2] < ret[2]:
                ret = [key1, key2, dm[key1][key2]]
    return ret

def tree_construction(dm, idlist, dc = "complete"):
    """
    Returns a TreeNode object denoting the current tree
    Precondition: 
        idlist: a list of motif ids. 
        dm: a np array representing the distance matrix. it should have a shape of len(idlist) * len(idlist)
        dc: the way for distance update method. Should be one of "complete", "single", "UPGMA", "WPGMA:
    """
    nodemap, dm = wrap_dm(idlist,dm)
    # need to build treenode for len(list(nodemap.keys()))-1 times
    for iter in range(len(list(nodemap.keys()))-1):
        m1, m2, sd = shortest(dm)
        
        # create the new node and update the nodemap
        new_node_name = iter
        left_node = nodemap[m1]
        right_node = nodemap[m2]
        lmn = left_node.mn
        rmn = right_node.mn
        mn = lmn + rmn
        height = sd / 2
        new_node = TreeNode(name = new_node_name, left_child = left_node, right_child = right_node, height = height, mn = mn)
        nodemap[new_node_name] = new_node 
        del nodemap[m1]
        del nodemap[m2]

        # update the distance matrix
        del dm[m1]
        del dm[m2]
        new_node_distances = {new_node_name : 0}
        for m in dm.keys():
            distance_l = dm[m][m1]
            distance_r = dm[m][m2]
            del dm[m][m1]
            del dm[m][m2]
            if dc == "single":
                new_distance = min(distance_l, distance_r)
            elif dc == "complete":
                new_distance = max(distance_l, distance_r)
            elif dc == "UPGMA":
                new_distance = (distance_l * lmn + distance_r * rmn ) / mn
            elif dc == "WPGMA":
                new_distance = (distance_l + distance_r) / 2
            dm[m][new_node_name] = new_distance
            new_node_distances[m] = new_distance
        dm[new_node_name] = new_node_distances
    assert len(list(nodemap.keys())) == 1
    return list(nodemap.values())[0]
            
        

def cut_tree(tree, nclusters):
    """
    Cut the given tree object to give nclusters
    Precondition: 
        tree is a TreeNode object
        nclusters is an integer >= 1
    """
    height_map = {tree.height : [tree]}
    remaining_steps = nclusters - 1
    while remaining_steps > 0:
        # find the highest node and split it into two child tree nodes
        # there might still be a need to handle the really rare case that actually two nodes are at the same height
        highest = max(list(height_map.keys()))
        highest_node = height_map[highest][0]
        if len(height_map[highest]) > 1:
            height_map[highest].pop(0)
        else:
            del height_map[highest]
        left_node = highest_node.left
        right_node = highest_node.right
        if left_node.height not in height_map: 
            height_map[left_node.height] = [left_node]
        else:
            height_map[left_node.height].append(left_node)
        if right_node.height not in height_map: 
            height_map[right_node.height] = [right_node]
        else:
            height_map[right_node.height].append(right_node)
        remaining_steps -= 1
    node_clusters = []
    for height in height_map.keys():
        node_clusters += height_map[height]
    assert len(node_clusters) == nclusters
    # now it is just to get the motif names from the TreeNode objects
    def degradenode(treenode):
        """
        helper function to degrade the nodes from the treeNode objects
        """
        if treenode.left == None:
            return [treenode.name]
        return degradenode(treenode.left) + degradenode(treenode.right)
    clusters = []
    for node in node_clusters:
        clusters.append(degradenode(node))
    return clusters
    

def hc(nclusters, dm, idlist, distance_update = "UPGMA"):
    """
    Grow a tree, then cut it!
    Preconditions:
        nclusters: an integer >= 1
        dm: np array for the distance matrix
        idlist: the motif ids
        distance_update: one of ["complete", "single", "UPGMA", "WPGMA"]
    """
    return cut_tree(tree_construction(dm, idlist, dc = distance_update), nclusters)
    

In [27]:
###############################################################################

# Dataset: Gapped vertebrates

dataset = gapped.DNABindingMotifs()
# dataset.mmm is a dictionary with format {"id": Bio.motifs.Motif object}
# where each motif matrix can be retrieved by motif.pssm

print(dataset.mmm)
print("Finish printing dataset \n")

dm, idlist = dist_from_col.calculate_distance_matrix(
    dataset, Pearson_CC_Distance, "expand", bg=[0.25, 0.25, 0.25, 0.25], average=np.mean
)
# dm, idlist = jensen_distance.calculate_distance_matrix(dataset)

dataset_cc = dataset.cc
print(dataset_cc)
print("Finish printing dataset cc \n")

# base_dir = "nature_clusters"
# generator = graph.MotifGraphGenerator(dataset_cc, base_dir)
# cluster_paths = generator.generate_cluster_graphs()
# generator.generate_final_composite_graph(cluster_paths)

# Convert dataset_cc into a list of lists, where each inner list contains the keys of the corresponding dictionary
dataset_cc_list = [[key for key in d] for d in dataset_cc]
print(dataset_cc_list)
print("Finish printing dataset cc list \n")

# K-means clustering
# One advantage of K-means is it can customize number of clusters needed
num_clusters = len(dataset_cc_list)  # Define the number of clusters
print(num_clusters)
print("Finish printing num_clusters \n")

clusters = hc(num_clusters, dm, idlist, distance_update="UPGMA")
print(clusters)
print("Finish printing result by K-means \n")

our_cluster = create_cluster_dicts(clusters, dataset)

print(our_cluster)
print("Finish printing our cluster \n")

# base_dir_our = "predicted_clusters"
# generator_our = graph.MotifGraphGenerator(our_cluster, base_dir_our)
# cluster_paths_our = generator_our.generate_cluster_graphs()
# generator_our.generate_final_composite_graph(cluster_paths_our)


# clusters = kmeans_self_defined_dist.kmeans_motifs(dm, idlist, num_clusters)


dataset_dic = dict(sorted(generate_labels_from_clusters(dataset_cc_list).items()))
print(dataset_dic)
print("Finish printing true labels \n")

clusters_dic = dict(sorted(generate_labels_from_clusters(clusters).items()))
print(clusters_dic)
print("Finish printing predicted labels \n")

true_labels = list(dataset_dic.values())
predicted_labels = list(clusters_dic.values())

score_ari = ari_score.evaluate_clustering(true_labels, predicted_labels)
score_fmi = fmi_score.evaluate_clustering(true_labels, predicted_labels)
score_nmi = nmi_score.evaluate_clustering(true_labels, predicted_labels)



This program runs on database: JASPAR2024
30.0motifs have been fetched.
there is no overlap between clusters. 
{'MA1712.2': <Bio.motifs.Motif object at 0x162fd0350>, 'MA2327.1': <Bio.motifs.Motif object at 0x17ec76f50>, 'MA2122.1': <Bio.motifs.Motif object at 0x17eca83d0>, 'MA1637.2': <Bio.motifs.Motif object at 0x17ec765d0>, 'MA0730.1': <Bio.motifs.Motif object at 0x17eb3fbd0>, 'MA0815.1': <Bio.motifs.Motif object at 0x17ec74190>, 'MA0106.3': <Bio.motifs.Motif object at 0x17eca9fd0>, 'MA0861.2': <Bio.motifs.Motif object at 0x17eb3c890>, 'MA0159.1': <Bio.motifs.Motif object at 0x17eb3f4d0>, 'MA2323.1': <Bio.motifs.Motif object at 0x17ec74050>, 'MA0811.2': <Bio.motifs.Motif object at 0x162fe17d0>, 'MA1643.2': <Bio.motifs.Motif object at 0x17ec11250>, 'MA0007.4': <Bio.motifs.Motif object at 0x17ec74810>, 'MA0810.2': <Bio.motifs.Motif object at 0x17ec740d0>, 'MA0813.1': <Bio.motifs.Motif object at 0x17ec76590>, 'MA0858.1': <Bio.motifs.Motif object at 0x17eb3c090>, 'MA0119.1': <Bio.motifs.

In [47]:
# ################################################################################

# # Dataset: fungi_large

dataset2 = fungi_large.DNABindingMotifs()
# dataset2.mmm is a dictionary with format {"id": Bio.motifs.Motif object}

print(dataset2.mmm)
print("Finish printing dataset2 \n")

dm2, idlist2 = dist_from_col.calculate_distance_matrix(
    dataset2, Euclidean_Distance, "expand", bg=[0.27, 0.23, 0.23, 0.27], average=np.mean
)

dataset2_cc = dataset2.cc
print(dataset2_cc)
print("Finish printing dataset2 cc \n")

# base_dir2 = "nature_clusters2"
# generator2 = graph.MotifGraphGenerator(dataset2_cc, base_dir2)
# cluster_paths2 = generator2.generate_cluster_graphs()
# generator2.generate_final_composite_graph(cluster_paths2)

# Convert dataset2_cc into a list of lists, where each inner list contains the keys of the corresponding dictionary
dataset2_cc_list = [[key for key in d] for d in dataset2_cc]
print(dataset2_cc_list)
print("Finish printing dataset2 cc list \n")



This program runs on database: JASPAR2024
229.0motifs have been fetched.
there is no overlap between clusters. 
{'MA0410.2': <Bio.motifs.Motif object at 0x130baa510>, 'MA0372.2': <Bio.motifs.Motif object at 0x130b97c10>, 'MA1437.2': <Bio.motifs.Motif object at 0x130b24390>, 'MA1434.1': <Bio.motifs.Motif object at 0x130bcd590>, 'MA0423.2': <Bio.motifs.Motif object at 0x130b97610>, 'UN0291.2': <Bio.motifs.Motif object at 0x130b54e90>, 'MA0278.2': <Bio.motifs.Motif object at 0x130b37310>, 'MA0342.1': <Bio.motifs.Motif object at 0x130b9fc50>, 'MA0374.2': <Bio.motifs.Motif object at 0x130b8f8d0>, 'MA0361.2': <Bio.motifs.Motif object at 0x130b25dd0>, 'MA0353.1': <Bio.motifs.Motif object at 0x130b69a90>, 'MA0344.1': <Bio.motifs.Motif object at 0x130b6a690>, 'MA0285.2': <Bio.motifs.Motif object at 0x130b27dd0>, 'MA0334.2': <Bio.motifs.Motif object at 0x130b26bd0>, 'MA0316.2': <Bio.motifs.Motif object at 0x130bb7410>, 'UN0089.2': <Bio.motifs.Motif object at 0x130b19150>, 'MA0400.2': <Bio.motifs

In [48]:
# Hierarchical clustering
# clusters = hierarchical_clustering.hierarchical_clustering(5, dm, idlist)
# # resulting cluters after using clustering method
# print(clusters)

num_clusters2 = len(dataset2_cc_list)  # Define the number of clusters
print(num_clusters2)
print("Finish printing num_clusters2 \n")

#clusters2 = hierarchical_clustering.hierarchical_clustering(num_clusters2, dm2, idlist2)
clusters2 = kmeans_hier_init.kmeans_motifs(dm2, idlist2, num_clusters2)
print(clusters2)
print("Finish printing clustering result \n")
our_cluster2 = create_cluster_dicts(clusters2, dataset2)

# base_dir_our2 = "predicted_clusters2"
# generator_our2 = graph.MotifGraphGenerator(our_cluster2, base_dir_our2)
# cluster_paths_our2 = generator_our2.generate_cluster_graphs()
# generator_our2.generate_final_composite_graph(cluster_paths_our2)


dataset2_dic = dict(sorted(generate_labels_from_clusters(dataset2_cc_list).items()))
print(dataset2_dic)
print("Finish printing true labels \n")

clusters2_dic = dict(sorted(generate_labels_from_clusters(clusters2).items()))
print(clusters2_dic)
print("Finish printing predicted labels \n")

true_labels2 = list(dataset2_dic.values())
predicted_labels2 = list(clusters2_dic.values())

score_ari2 = ari_score.evaluate_clustering(true_labels2, predicted_labels2)
score_fmi2 = fmi_score.evaluate_clustering(true_labels2, predicted_labels2)
score_nmi2 = nmi_score.evaluate_clustering(true_labels2, predicted_labels2)

61
Finish printing num_clusters2 

[['UN0081.2', 'MA0373.1', 'MA0333.2', 'MA0394.1', 'MA0332.1'], ['UN0281.2', 'MA0303.3', 'MA0272.2'], ['MA0382.3', 'MA0286.2', 'MA0416.2'], ['MA0343.2', 'MA0327.1'], ['MA0397.2', 'MA0396.2', 'MA0384.2'], ['UN0091.2'], ['MA0307.1', 'MA0289.1', 'MA0300.2', 'MA0312.3', 'MA0309.2'], ['UN0302.1', 'UN0096.1'], ['UN0089.2', 'MA0355.2', 'MA0385.2'], ['MA0266.2'], ['MA0426.1', 'MA0433.2'], ['MA0417.1', 'MA0406.2'], ['UN0297.2', 'MA0296.2', 'MA0297.1', 'UN0092.1', 'MA0317.2', 'MA0929.2'], ['MA0435.2', 'MA0434.2'], ['MA0404.1', 'MA0349.2'], ['MA0285.2', 'MA0334.2', 'UN0093.2', 'UN0274.2', 'UN0094.2', 'UN0082.2', 'MA0368.1', 'UN0099.1', 'UN0103.2', 'UN0095.2'], ['MA0407.1'], ['MA0365.2'], ['MA0440.1'], ['MA0359.3'], ['MA0290.1'], ['MA0323.1'], ['MA0411.2'], ['MA0369.2'], ['MA0305.2'], ['MA0278.2', 'MA0318.1', 'MA0408.2'], ['MA0400.2', 'MA0324.1', 'UN0284.2', 'MA0280.2', 'MA0308.2', 'MA0358.2', 'MA0299.1', 'UN0283.2', 'MA0380.2', 'UN0299.1', 'UN0280.2', 'MA0275.1',

In [15]:
###############################################################################

Dataset: fungi

dataset3 = fungi.DNABindingMotifs()
# dataset3.mmm is a dictionary with format {"id": Bio.motifs.Motif object}

print(dataset3.mmm)
print("Finish printing dataset3 \n")

dm3, idlist3 = dist_from_col.calculate_distance_matrix(
    dataset3, Kullback_Leibler_Distance, "expand", bg=[0.27, 0.23, 0.23, 0.27], average=np.mean
)

dataset3_cc = dataset3.cc
print(dataset3_cc)
print("Finish printing dataset3 cc \n")

# base_dir3 = "nature_clusters3"
# generator3 = graph.MotifGraphGenerator(dataset3_cc, base_dir3)
# cluster_paths3 = generator3.generate_cluster_graphs()
# generator3.generate_final_composite_graph(cluster_paths3)

This program runs on database: JASPAR2024
178.0motifs have been fetched.
there is no overlap between clusters. 
{'MA1432.1': <Bio.motifs.Motif object at 0x17ecac990>, 'MA0364.1': <Bio.motifs.Motif object at 0x17ecb6390>, 'MA0395.1': <Bio.motifs.Motif object at 0x16eca6590>, 'MA0391.1': <Bio.motifs.Motif object at 0x16ebd8090>, 'MA0439.1': <Bio.motifs.Motif object at 0x16ebd19d0>, 'MA1436.1': <Bio.motifs.Motif object at 0x16ebd3010>, 'MA0323.1': <Bio.motifs.Motif object at 0x16ec95250>, 'MA0269.1': <Bio.motifs.Motif object at 0x16eca6150>, 'MA0420.1': <Bio.motifs.Motif object at 0x16ebda550>, 'MA0281.2': <Bio.motifs.Motif object at 0x16ec86790>, 'MA0929.1': <Bio.motifs.Motif object at 0x16ec7a510>, 'MA0389.1': <Bio.motifs.Motif object at 0x17ecae7d0>, 'MA0414.1': <Bio.motifs.Motif object at 0x16ec8ce90>, 'MA0404.1': <Bio.motifs.Motif object at 0x16ec8dad0>, 'MA0320.1': <Bio.motifs.Motif object at 0x16ec7af50>, 'MA0390.1': <Bio.motifs.Motif object at 0x16ec94610>, 'MA0361.1': <Bio.motifs

In [17]:


# Convert dataset3_cc into a list of lists, where each inner list contains the keys of the corresponding dictionary
dataset3_cc_list = [[key for key in d] for d in dataset3_cc]
print(dataset3_cc_list)
print("Finish printing dataset3 cc list \n")


num_clusters3 = len(dataset3_cc_list)  # Define the number of clusters
print(num_clusters3)
print("Finish printing num_clusters3 \n")

cluster3 = hc(num_clusters3, dm3, idlist3, distance_update="UPGMA")
print(cluster3)
print("Finish printing result by hierarchical clustering \n")


# cluster3 = kmeans_hier_init.kmeans_motifs(dm3, idlist3, num_clusters3)
# print(cluster3)
# print("Finish printing result by K means \n")

our_cluster3 = create_cluster_dicts(cluster3, dataset3)

# base_dir_our3 = "predicted_clusters3"
# generator_our3 = graph.MotifGraphGenerator(our_cluster3, base_dir_our3)
# cluster_paths_our3 = generator_our3.generate_cluster_graphs()
# generator_our3.generate_final_composite_graph(cluster_paths_our3)


dataset3_dic = dict(sorted(generate_labels_from_clusters(dataset3_cc_list).items()))
print(dataset3_dic)
print("Finish printing true labels \n")

clusters3_dic = dict(sorted(generate_labels_from_clusters(cluster3).items()))
print(clusters3_dic)
print("Finish printing predicted labels \n")

true_labels3 = list(dataset3_dic.values())
predicted_labels3 = list(clusters3_dic.values())

score_ari3 = ari_score.evaluate_clustering(true_labels3, predicted_labels3)
score_fmi3 = fmi_score.evaluate_clustering(true_labels3, predicted_labels3)
score_nmi3 = nmi_score.evaluate_clustering(true_labels3, predicted_labels3)
    



print(score_ari3)
print(score_fmi3)
print(score_nmi3)


[['MA0268.1', 'MA0306.1', 'MA0337.1', 'MA0338.1', 'MA0339.1', 'MA0341.1', 'MA0342.1', 'MA0364.1', 'MA0366.1', 'MA0372.1', 'MA0423.1', 'MA0431.1', 'MA0436.1', 'MA0441.1', 'MA1431.1', 'MA1432.1', 'MA1433.1', 'MA1434.1'], ['MA0293.1', 'MA0301.1', 'MA0302.1', 'MA0389.1', 'MA0434.1', 'MA1437.1'], ['MA0384.1', 'MA0396.1', 'MA0397.1'], ['MA0318.1', 'MA0321.1', 'MA0322.1', 'MA0328.2', 'MA0408.1'], ['MA0350.1', 'MA0351.1'], ['MA0275.1', 'MA0280.1', 'MA0282.1', 'MA0292.1', 'MA0294.1', 'MA0311.1', 'MA0325.1', 'MA0348.1', 'MA0354.1', 'MA0360.1', 'MA0367.1', 'MA0380.1', 'MA0391.1', 'MA0392.1', 'MA0420.1', 'MA0422.1', 'MA0424.1', 'MA0428.1', 'MA0430.1', 'MA0432.1', 'MA0437.1', 'MA0438.1', 'MA0439.1'], ['MA0310.1', 'MA0357.1', 'MA0409.1', 'MA1436.1'], ['MA0285.1', 'MA0361.1', 'MA0381.1', 'MA0429.1'], ['MA0295.1', 'MA0329.1', 'MA0330.1', 'MA0401.1'], ['MA0332.1', 'MA0333.1', 'MA0334.1', 'MA0373.1'], ['MA0267.2', 'MA0402.2'], ['MA0289.1', 'MA0300.1', 'MA0307.1', 'MA0309.1', 'MA0312.2'], ['MA0355.1', 'M

In [50]:
###############################################################################

Dataset: vertebrates_large

dataset4 = vertebrates_large.DNABindingMotifs()
# dataset4.mmm is a dictionary with format {"id": Bio.motifs.Motif object}

print(dataset4.mmm)
print("Finish printing dataset4 \n")

dm4, idlist4 = dist_from_col.calculate_distance_matrix(
    dataset4, Euclidean_Distance, "expand", bg=[0.25, 0.25, 0.25, 0.25], average=np.mean
)

dataset4_cc = dataset4.cc
print(dataset4_cc)
print("Finish printing dataset4 cc \n")

# base_dir4 = "nature_clusters4"
# generator4 = graph.MotifGraphGenerator(dataset4_cc, base_dir4)
# cluster_paths4 = generator4.generate_cluster_graphs()
# generator4.generate_final_composite_graph(cluster_paths4)

This program runs on database: JASPAR2024
1196.0motifs have been fetched.
there is no overlap between clusters. 
{'MA1116.2': <Bio.motifs.Motif object at 0x131091ad0>, 'MA0692.2': <Bio.motifs.Motif object at 0x1310b4e10>, 'MA0666.3': <Bio.motifs.Motif object at 0x1311bd190>, 'MA2334.1': <Bio.motifs.Motif object at 0x1311d7f50>, 'MA2095.1': <Bio.motifs.Motif object at 0x13105c050>, 'MA0047.4': <Bio.motifs.Motif object at 0x130fb3950>, 'MA0100.4': <Bio.motifs.Motif object at 0x13100a410>, 'UN0652.2': <Bio.motifs.Motif object at 0x1313eadd0>, 'MA0864.3': <Bio.motifs.Motif object at 0x13127bd10>, 'MA2324.1': <Bio.motifs.Motif object at 0x13112be90>, 'MA0160.3': <Bio.motifs.Motif object at 0x1311394d0>, 'MA0083.3': <Bio.motifs.Motif object at 0x13131b0d0>, 'MA1998.2': <Bio.motifs.Motif object at 0x131363890>, 'MA1963.2': <Bio.motifs.Motif object at 0x1311952d0>, 'MA1497.2': <Bio.motifs.Motif object at 0x1311bec10>, 'MA0828.3': <Bio.motifs.Motif object at 0x1310acfd0>, 'MA1529.2': <Bio.motif

In [56]:


# Convert dataset4_cc into a list of lists, where each inner list contains the keys of the corresponding dictionary
dataset4_cc_list = [[key for key in d] for d in dataset4_cc]
print(dataset4_cc_list)
print("Finish printing dataset4 cc list \n")


num_clusters4 = len(dataset4_cc_list)  # Define the number of clusters
print(num_clusters4)
print("Finish printing num_clusters4 \n")

# cluster4 = hierarchical_clustering.hierarchical_clustering(num_clusters4, dm4, idlist4)
# print(cluster4)
# print("Finish printing result by hierarchical clustering \n")


cluster4 = kmeans_hier_init.kmeans_motifs(dm4, idlist4, num_clusters4)
print(cluster4)
print("Finish printing result by K means \n")

our_cluster4 = create_cluster_dicts(cluster4, dataset4)

# base_dir_our3 = "predicted_clusters3"
# generator_our3 = graph.MotifGraphGenerator(our_cluster3, base_dir_our3)
# cluster_paths_our3 = generator_our3.generate_cluster_graphs()
# generator_our3.generate_final_composite_graph(cluster_paths_our3)


dataset4_dic = dict(sorted(generate_labels_from_clusters(dataset4_cc_list).items()))
print(dataset4_dic)
print("Finish printing true labels \n")

clusters4_dic = dict(sorted(generate_labels_from_clusters(cluster4).items()))
print(clusters4_dic)
print("Finish printing predicted labels \n")

true_labels4 = list(dataset4_dic.values())
predicted_labels4 = list(clusters4_dic.values())

score_ari4 = ari_score.evaluate_clustering(true_labels4, predicted_labels4)
score_fmi4 = fmi_score.evaluate_clustering(true_labels4, predicted_labels4)
score_nmi4 = nmi_score.evaluate_clustering(true_labels4, predicted_labels4)
    



print(score_ari4)
print(score_fmi4)
print(score_nmi4)


[['MA1592.2', 'MA1928.2', 'MA0835.3', 'MA0462.3', 'MA1634.2', 'MA1988.2', 'MA1101.3', 'MA0476.2', 'MA0489.3', 'MA1132.2', 'MA1142.2', 'MA1134.2', 'MA1144.2', 'MA1135.2', 'MA1138.2', 'MA1128.2', 'MA1141.2', 'MA1137.2', 'MA0099.4', 'MA1130.2', 'MA1633.2', 'MA0655.1', 'MA0477.3', 'MA0490.3', 'MA0491.3', 'MA0478.2', 'MA0841.2', 'MA1520.2', 'MA1521.2', 'MA0496.4', 'MA0089.3', 'MA0150.3', 'MA0501.2', 'MA0591.2', 'MA0659.4', 'MA0738.2', 'MA0739.2', 'MA0161.3', 'MA0670.2', 'MA0671.2', 'MA0498.3', 'MA0774.1', 'MA0775.2', 'MA0782.3', 'MA1114.2', 'UN0799.1', 'UN0673.2', 'UN0807.1', 'UN0570.2'], ['MA1643.2', 'MA0119.1', 'MA1528.2', 'MA1527.2', 'UN0127.2', 'UN0666.2', 'UN0621.2', 'UN0600.2', 'UN0629.2'], ['MA0041.3', 'MA0847.4', 'MA1487.3', 'MA1607.2', 'MA0846.2', 'MA0032.2', 'MA0845.1', 'MA0635.2', 'MA0877.4', 'MA0040.2', 'MA0047.4', 'MA2118.1', 'MA0850.1', 'MA0848.1', 'MA0849.1', 'MA0031.2', 'MA0480.3', 'MA0157.4', 'MA0852.3', 'MA0042.2', 'MA2117.1', 'MA1103.3', 'MA0481.4', 'MA1683.2', 'MA0033.2'